# MTH3302 - Méthodes probabilistes et statistiques pour I.A.
#### Polytechnique Montréal
### Projet A2024
----
Équipe T - TODO
### Objectif
Prédiction de **la consommation en carburant de voitures récentes**.

### Données
Le jeu de données contient pour presque 400 véhicule, la consommation moyenne en L/100km, l'année de frabrication, le type de véhicule, le nombre de cylindre, cylindree, la transmission et la boite.

- `train.csv` est l'ensemble d'entraînement
- `test.csv` est l'ensemble de test


## Chargement des données

Importation des librairies utilisées dans le calepin.

// TODO: à enlever à la fin:

Pour importer librairies:
using Pkg
Pkg.add(["CSV", "DataFrames", "Combinatorics", "Gadfly", "Distributions"], ...)

In [26]:
import Pkg; Pkg.add("Plots")

using CSV, DataFrames, Statistics, Dates, Gadfly, LinearAlgebra, Plots

   Resolving package versions...
   Installed x265_jll ───────────────────── v3.5.0+0
   Installed GR_jll ─────────────────────── v0.73.8+0
   Installed libdecor_jll ───────────────── v0.2.2+0
   Installed JpegTurbo_jll ──────────────── v3.0.4+0
   Installed Libmount_jll ───────────────── v2.40.2+0
   Installed libfdk_aac_jll ─────────────── v2.0.3+0
   Installed LoggingExtras ──────────────── v1.1.0
   Installed LERC_jll ───────────────────── v4.0.0+0
   Installed Opus_jll ───────────────────── v1.3.3+0
   Installed Xorg_xkbcomp_jll ───────────── v1.4.6+0
   Installed RelocatableFolders ─────────── v1.0.1
   Installed Unitful ────────────────────── v1.21.1
   Installed Xorg_xcb_util_wm_jll ───────── v0.4.1+1
   Installed ConcurrentUtilities ────────── v2.4.2
   Installed Xorg_xcb_util_image_jll ────── v0.4.0+1
   Installed PlotUtils ──────────────────── v1.4.3
   Installed RecipesPipeline ────────────── v0.6.12
   Installed OpenSSL ────────────────────── v1.4.3
   Installed Xorg_libpt

In [4]:
function standardize(data)
    return (data .- mean(data)) ./ std(data)
end

function standardize_data(data)
    stddata = deepcopy(data)
   for col in names(stddata)
        if eltype(stddata[!, col]) <: Number
            stddata[!, col] = standardize(stddata[!, col])
        end
    end
    return stddata
end

standardize_data (generic function with 1 method)

In [5]:
function encode(data, column)
    for c in unique(data[!, column])
        data[!, Symbol(c)] = ifelse.(data[!, column] .== c, 1, 0)
    end
    return data
end

function encode_data(data)
    encoded_data = deepcopy(data)
    encoded_data = encode(encoded_data, :type)
    encoded_data = encode(encoded_data, :transmission)
    encoded_data = encode(encoded_data, :boite)
    return encoded_data
end

encode_data (generic function with 1 method)

In [6]:
# faire une méthode pour les outliers

# code pas tester:
# function remove_outliers(data)
#     return data[(abs.(zscore(data)) .< 3) .| isnan.(zscore(data)), :]
# end

function get_outliers(data)
    outliers = DataFrame()
    cleaned_data = DataFrame()
    data = data[:, [:consommation, :volume_gaz]]

    for col in names(data)
        if eltype(data[!, col]) <: Number
            Q1 = quantile(data[!, col], 0.20)
            Q4 = quantile(data[!, col], 0.80)
            IQR = Q4 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q4 + 1.5* IQR
            println("col: ", col, " lower: ", lower_bound, " upper: ", upper_bound, " Q1: ", Q1, " Q4: ", Q4, " IQR: ", IQR)
            col_outliers = data[(data[!, col] .< lower_bound) .| (data[!, col] .> upper_bound), :]
            outliers = vcat(outliers, col_outliers)
        end
    end
    return unique(outliers)
end

get_outliers (generic function with 1 method)

In [7]:
function removeRows(data)
    return select!(data, Not([:type, :transmission, :boite]))
    # return select!(data, Not([:annee]))
end

removeRows (generic function with 1 method)

In [8]:
function concatenate(data)
    data[!, :typecar] = data[!, :type] .* data[!, :transmission] .* data[!, :boite]
    return removeRows(data)
end

concatenate (generic function with 1 method)

In [9]:
function getStandardEncodedData(data)
    data_copy = deepcopy(data)
    standardised_data = standardize_data(data_copy)
    # standardised_data = encode_data(standardised_data)
    # standardised_data = removeRows(standardised_data)
    standardised_data = concatenate(standardised_data)
    standardised_data = encode(standardised_data, :typecar)
    select!(standardised_data, Not([:typecar]))
    return standardised_data
end




getStandardEncodedData (generic function with 1 method)

In [10]:
function rmse(y, ychap)
    return sqrt(mean((ychap .- y).^2))
end


rmse (generic function with 1 method)

In [11]:
function rsquared(y, ychap)
    ss_total = sum((y .- mean(y)).^2)
    ss_res = sum((y .- ychap).^2)
    return ss_res / ss_total
end


rsquared (generic function with 1 method)

In [12]:
function split_data(data,k)
    n = size(data, 1)
    validation_size = floor(Int, n * k)

    validation_indices = rand(1:n, validation_size)  
    
    validation_data = data[validation_indices, :]
    training_data = data[setdiff(1:n, validation_indices), :]
    
    return training_data, validation_data
end

split_data (generic function with 1 method)

In [13]:
function regression(standardised_data, trainTestSplitPercentage)
    training_data, test_data = split_data(standardised_data, trainTestSplitPercentage)
    X_train =  Matrix(training_data[:, Not(:consommation)])
    y_train = training_data[:, :consommation]

    beta = X_train \ y_train

    rmseval = 0.0
    if trainTestSplitPercentage != 0.0
        X_test = Matrix(test_data[:, Not(:consommation)])
        y_test = test_data[:, :consommation]
        ychap =  X_test * beta
        ychap = (ychap .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
        y_test = (y_test .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
        # print("regression\n")
        # print("r2 = ", rsquared(y_test, ychap), "\n")
        rmseval = rmse(y_test, ychap)
        # print("rmse = ", rmseval , "\n\n")
    end
    
    return rmseval, beta
end


regression (generic function with 1 method)

In [14]:
function ridge_regression(standardised_data, lambda, trainTestSplitPercentage)
    training_data, test_data = split_data(standardised_data, trainTestSplitPercentage)
    X_train = Matrix(training_data[:, Not([:consommation])])
    y_train = training_data[:, :consommation]
    beta = (X_train'X_train + lambda*I)\X_train'y_train

    rmseval = 0.0
    if trainTestSplitPercentage != 0.0
        X_test = Matrix(test_data[:, Not(:consommation)])
        y_test = test_data[:, :consommation]
        ychap =  X_test * beta
        ychap = (ychap .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
        y_test = (y_test .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
        # print("ridge: \n")
        # print("r2 = ", rsquared(y_test, ychap), "\n")
        rmseval = rmse(y_test, ychap)
        # print("rmse = ", rmseval , "\n\n")
    end
    return rmseval, beta
end

ridge_regression (generic function with 1 method)

In [15]:
# pas tester: (aucune idee si ca marche)

function svd_regression(standardised_data, trainTestSplitPercentage)
    training_data, test_data = split_data(standardised_data, trainTestSplitPercentage)
    X_train = Matrix(training_data[:, Not([:consommation])])
    y_train = training_data[:, :consommation]
    
    U, S, V = svd(X_train)

    # beta = V * (Diagonal(1 ./ S) * U' * y_train)

    beta = V' * Diagonal([s > 1e-10 ? 1/s : 0 for s in S]) * U' * y_train

    rmseval = 0.0
    if trainTestSplitPercentage != 0.0
        X_test = Matrix(test_data[:, Not(:consommation)])
        y_test = test_data[:, :consommation]
        ychap =  X_test * beta
        ychap = (ychap .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
        y_test = (y_test .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
        # print("regression\n")
        # print("r2 = ", rsquared(y_test, ychap), "\n")
        rmseval = rmse(y_test, ychap)
        # print("rmse = ", rmseval , "\n\n")
    end
    return rmseval,beta
end

svd_regression (generic function with 1 method)

In [16]:
trainData = CSV.read("./data/train.csv", DataFrame)
testData = CSV.read("./data/test.csv", DataFrame)
first(testData, 1)

Row,annee,type,nombre_cylindres,cylindree,transmission,boite
,Int64,String31,Int64,String3,String15,String15
1,2014,voiture_moyenne,4,"2,5",traction,manuelle


In [17]:
trainData.consommation = parse.(Float64,replace.(trainData.consommation, "," => "."))
trainData.cylindree = parse.(Float64,replace.(trainData.cylindree, "," => "."))
testData.cylindree = parse.(Float64,replace.(testData.cylindree, "," => "."))


150-element Vector{Float64}:
 2.5
 2.5
 2.5
 2.0
 5.8
 5.0
 5.0
 2.4
 3.5
 5.2
 5.2
 5.9
 2.0
 ⋮
 1.6
 3.3
 5.0
 2.0
 2.0
 1.6
 2.0
 2.0
 3.0
 3.0
 1.5
 2.0

In [23]:
trainData[!,:volume_gaz] = trainData[!,:nombre_cylindres] .* trainData[!,:cylindree]
testData[!,:volume_gaz] = testData[!,:nombre_cylindres] .* testData[!,:cylindree]
first(trainData, 1)
COMSOMMATION_MEAN = mean(trainData.consommation)
COMSOMMATION_STD = std(trainData.consommation)


2.139763088813657

In [19]:
data = getStandardEncodedData(trainData)
# print(first(data, 1))
nrange = 1000
n = 0
for i in range(0, 1, length=nrange)
    n += regression(data, 0.2)[1]
end
print("average rmse for regression: ", n/nrange, "\n")

n = 0
for i in range(0, 1, length=nrange)
    r = ridge_regression(data, 10, 0.2)[1]
    # if r > 1.7
    #     print("ridge rmse: ", r, "\n")
    # end
    n += r
    
end
print("average rmse for ridge: ", n/nrange, "\n")

# n = 0
# for i in range(0, 1, length=nrange)
#     n += svd_regression(data, 0.05)[1]
# end
# print("average rmse for svd: ", n/nrange, "\n")


average rmse for regression: 0.9193419539346555
average rmse for ridge: 0.9665463685564598


In [20]:
function linear_regression(x, y)
    n = length(x)
    if n == 0
        error("Cannot compute linear regression with zero elements.")
    end
    x_mean = mean(x)
    y_mean = mean(y)
    slope = sum((x .- x_mean) .* (y .- y_mean)) / sum((x .- x_mean).^2)
    intercept = y_mean - slope * x_mean
    return slope, intercept
end

linear_regression (generic function with 1 method)

In [21]:
function get_outliers_columns(data, column_name; threshold=1.5)
    Q1 = quantile(data[!, column_name], 0.25)
    Q3 = quantile(data[!, column_name], 0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - threshold * IQR
    upper_bound = Q3 + threshold * IQR
    return data[(data[!, column_name] .< lower_bound) .| (data[!, column_name] .> upper_bound), :]
end

get_outliers_columns (generic function with 1 method)

In [24]:
for vehicle_type in unique(trainData.type)
    println(vehicle_type)
    
    # Filter data by type
    data_type = trainData[trainData.type .== vehicle_type, :]
    
    if nrow(data_type) < 2
        println("Not enough data for vehicle type: $vehicle_type")
        continue
    end
    
    # Linear regression line parameters
    slope, intercept = linear_regression(data_type.volume_gaz, data_type.consommation)
    regression_line_y = intercept .+ slope .* data_type.volume_gaz
    
    # Residuals and margin for bounds
    residuals = data_type.consommation .- regression_line_y
    margin = std(residuals)
    
    # Define max and min lines
    max_line_y = regression_line_y .+ margin
    min_line_y = regression_line_y .- margin
    
    # Detect outliers based on IQR
    outliers = get_outliers_columns(data_type, :consommation)
    
    # Plotting
    scatter(data_type.volume_gaz, data_type.consommation, label="Data", legend=:top, color=:blue, alpha=0.6)
    scatter!(outliers.volume_gaz, outliers.consommation, label="Outliers", color=:red, alpha=0.9)
    plot!(data_type.volume_gaz, regression_line_y, label="Regression Line", color=:black)
    plot!(data_type.volume_gaz, max_line_y, label="Max Bound", linestyle=:dash, color=:green)
    plot!(data_type.volume_gaz, min_line_y, label="Min Bound", linestyle=:dash, color=:red)
    
    xlabel!("Volume Gaz")
    ylabel!("Consommation")
    title!("Regression Line and Outliers for Vehicle Type: $vehicle_type")
    display(plot())
    
    # Removing outliers and points outside max and min bounds
    cleaned_data = data_type[
        (data_type.consommation .>= min_line_y) .&
        (data_type.consommation .<= max_line_y) .&
        .!in.(axes(data_type, 1), axes(outliers, 1)), :
    ]
    
    # Saving the cleaned data
    CSV.write("cleaned_data_$vehicle_type.csv", cleaned_data)
    println("Cleaned data for $vehicle_type saved to cleaned_data_$vehicle_type.csv")
end

voiture_moyenne


LoadError: UndefVarError: `scatter` not defined

In [ ]:
test_data = getStandardEncodedData(testData)
train_data = getStandardEncodedData(trainData)

println("Columns in test_data but not in train_data: ", setdiff(names(test_data), names(train_data)))
println("Columns in train_data but not in test_data: ", setdiff(names(train_data), names(test_data)))
# println(size(testData))
# println(size(trainData))
# println(size(test_data))
# println(size(train_data))


common_columns = intersect(names(test_data), names(train_data))
select!(train_data, common_columns)


beta = ridge_regression(train_data, 10, 0.0)[2]

common_columns = intersect(names(test_data), names(train_data))
# TODO retirer les valeurs qui ne sont pas dans lintersection??
print("common columns: ", size(common_columns), "\n")
X_test = Matrix(select(test_data, common_columns))
println(size(X_test))
println(size(test_data))
 println(size(beta))

ychap =  X_test * beta

ychap = (ychap .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
# # print(ychap)
id = 1:150
df_pred = DataFrame(id=id, consommation=ychap)

# using Dates
# current_time = Dates.format(now(), "yyyy-mm-dd_HH-MM-SS")
# file_name = "benchmark_" * current_time * ".csv"
# CSV.write("./soumissions_potentielles/" *file_name, df_pred)

Columns in test_data but not in train_data: ["camionnette_standardintegraleautomatique", "break_petittractionmanuelle", "camionnette_petit4x4automatique"]
Columns in train_data but not in test_data: ["consommation", "camionnette_petitpropulsionautomatique", "voiture_compactepropulsionmanuelle", "voiture_minicompactepropulsionautomatique", "voiture_sous_compacteintegralemanuelle", "break_petitintegraleautomatique", "camionnette_petittractionautomatique", "VUS_petitintegralemanuelle", "VUS_petit4x4manuelle"]


LoadError: ArgumentError: column name :consommation not found in the data frame

In [ ]:
# for type in unique(trainData.type)
#     println(type)
#     data_type = trainData[trainData.type .== type, :]
#     println(combine(groupby(data_type, :transmission), :consommation => mean, :volume_gaz => mean, nrow => :nrow))
#     println()
# end

In [ ]:
# for type in unique(trainData.type)
#     println(type)
#     data_type = trainData[trainData.type .== type, :]
#     display(plot(x=data_type.volume_gaz, y=data_type.consommation))
#     println()
# end

## Exploration des données